In [1]:
# ========================================
# 📌 1. Instalación de dependencias
# ========================================
!pip install langchain-community faiss-cpu pypdf python-docx unstructured -q
!pip install msoffcrypto-tool
# Instalar la librería jq necesaria para JSONLoader
!pip install jq -q

import json
import importlib.metadata    # Importación del modulo de metadata de la libreria importlib
import warnings               # Importación de la libreria warnings
import pandas as pd

# Uso de la libreria importlib.metadata para obtener los paquetes
# instalados en el Notebook
installed_packages = [dist.metadata['Name'].lower() for dist in importlib.metadata.Distribution.discover()]

# Chequeando que Google Colab este entre los paquetes instalados
# para confirmar si es el ambiente de trabajo usado.
IN_COLAB = 'google-colab' in installed_packages

# Ignorar algun Warning generado durante la ejecución del Notebook
warnings.filterwarnings('ignore')

# Instalar Ollama solo si no está ya instalado
!if ! type ollama > /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; else echo "Ollama ya está instalado."; fi

# Iniciar Ollama en segundo plano (con logs en ollama.log)
!nohup ollama serve > ollama.log 2>&1 &

# Descargar los modelos necesarios
!ollama pull llama3.2:3b
!ollama pull nomic-embed-text

Ollama ya está instalado.




In [2]:
# ========================================
# 📌 2. Carga de Documentos
# ========================================
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader, UnstructuredExcelLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import drive
from langchain.schema import Document # Import Document class


pdf_loader = PyPDFLoader('/content/drive/MyDrive/Ecomarket/Política de Devoluciones.pdf') # ruta de archivo PDF
ruta_json = '/content/drive/MyDrive/Ecomarket/faq.json' # ruta archivo JSON
# Cargar archivo JSON. Usando jq_schema='.' para cargar el contenido completo.
# Si el JSON es una lista de objetos.
# Usamos text_content=False para evitar que JSONLoader valide si el contenido es string al cargar.
json_loader = JSONLoader(file_path=ruta_json, jq_schema='.', text_content=False)
# Cargar Excel con contexto semántico
ruta_excel = '/content/drive/MyDrive/Ecomarket/Inventario_Sostenible.xlsx'
df = pd.read_excel(ruta_excel)

excel_docs = []
for i, row in df.iterrows():
    content = (
        f"Producto: {row['Nombre del Producto']}\n"
        f"Categoría: {row['Categoría']}\n"
        f"Cantidad en stock: {row['Cantidad en Stock']}\n"
        f"Precio unitario: {row['Precio Unitario ($)']} dólares\n"
        f"Fecha de ingreso: {row['Fecha de Ingreso']}\n"
    )
    excel_docs.append(Document(page_content=content, metadata={"source": "Inventario_Sostenible.xlsx"}))


# Cargar documentos de cada fuente
pdf_docs = pdf_loader.load()

json_output = json_loader.load()
json_docs = []

if not isinstance(json_output, list):
    json_output = [json_output]

for item in json_output:
    if isinstance(item, dict):  # Asegurarse de que sea un diccionario
        pregunta = item.get("pregunta", "")
        respuesta = item.get("respuesta", "")
        content = f"Pregunta frecuente: {pregunta}\nRespuesta: {respuesta}"
    else:
        content = str(item)  # Fallback si no es dict
    json_docs.append(Document(page_content=content, metadata={"source": "faq.json"}))


docs = pdf_docs + json_docs + excel_docs

# Segmentación en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Documentos cargados y segmentados en {len(chunks)} chunks")

Documentos cargados y segmentados en 19 chunks


In [3]:
# ========================================
# 📌 3. Embeddings + Base de Datos Vectorial
# ========================================


from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Crear embeddings con Ollama
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Indexar en FAISS
db = FAISS.from_documents(chunks, embeddings)

# Guardar en disco (opcional)
db.save_local("faiss_ecoshop")


/tmp/ipython-input-740107090.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [4]:
# ========================================
# 📌 4. Construcción del RAG
# ========================================
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

# LLM con Ollama
llm = Ollama(model="llama3.2:3b")

# Crear el pipeline RAG
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

print(" Pipeline RAG listo")



/tmp/ipython-input-3888843894.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:3b")
/tmp/ipython-input-3888843894.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = qa.run("¿Cuánto tiempo tengo para devolver un producto que no he usado?")


 Pipeline RAG listo
 Respuesta: Puedes devolver el producto dentro de los 30 días después de la fecha de compra.


In [5]:
# ========================================
# 📌 5. Pruebas del Sistema RAG
# ========================================

preguntas = [
  {
    "pregunta": "¿Qué productos de EcoMarket tienen un precio menor a 10 dólares?"
  },
  {
    "pregunta": "¿Cuáles son los productos disponibles en la categoría “Electrónica”?"
  },
  {
    "pregunta": "¿Qué producto tiene el menor stock actualmente?"
  },
  {
    "pregunta": "¿Hay productos ingresados después del 20 de septiembre de 2025?"
  },
  {
    "pregunta": "¿Qué productos están disponibles para oficina?"
  },
  {
    "pregunta": "¿Cuáles son los productos biodegradables o ecológicos que ofrece EcoMarket?"
  },
  {
    "pregunta": "¿Qué opciones de iluminación sostenible ofrece EcoMarket?"
  },
  {
    "pregunta": "¿Qué productos están hechos de bambú?"
  },
  {
    "pregunta": "¿Cuál es el producto más caro del inventario?"
  },
  {
    "pregunta": "¿Qué productos podrían considerarse como regalos sostenibles?"
  }
]


for p in preguntas:
    print(f"\n🔹 Pregunta: {p['pregunta']}") # Access the question string directly
    respuesta = qa.run(p['pregunta']) # Pass the question string to qa.run()
    print(f" Respuesta: {respuesta}")


🔹 Pregunta: ¿Qué productos de EcoMarket tienen un precio menor a 10 dólares?
 Respuesta: No tengo suficiente información para determinar qué productos de EcoMarket tienen un precio menor a 10 dólares. La pregunta no incluye los precios de todos los productos disponibles en EcoMarket, por lo que no puedo proporcionar una respuesta precisa. Sin embargo, puedo decir que el Bolsa de Tela Ecológica Reutilizable tiene un precio unitario de 3.5 dólares, que es menor a 10 dólares.

🔹 Pregunta: ¿Cuáles son los productos disponibles en la categoría “Electrónica”?
 Respuesta: Los productos disponibles en la categoría "Electrónica" son:

1. Cargador de Bicicleta USB (Energía Cinética)
2. Cargador Solar Portátil

🔹 Pregunta: ¿Qué producto tiene el menor stock actualmente?
 Respuesta: No puedo proporcionar una respuesta sin más información, pero puedo decir que la pregunta requiere comparar los niveles de inventario de diferentes productos. Sin embargo, no tengo suficiente información para determin